Guassian
---
Uni-model

$$ f(x) = \frac{1}{\sqrt{2\pi\sigma^2}}\exp[-\frac{(x-\mu)^2}{2\sigma^2}] $$

In [1]:
import numpy as np

Bayes Rule
---
Measurements is gaining information
<img src="pics/bayes.png" width=400>

### Multiplication of Guassian(Measurement Update Procedure)
---
$$ N_1(\mu, \sigma^2)$$
$$ N_2(\gamma, r^2) $$

$$ \mu_{new} = \frac{1}{\sigma^2 + r^2}(r^2\mu + \sigma^2\gamma)$$

$$ \sigma_{new} = 1/[{\frac{1}{\sigma^2} + \frac{1}{\gamma^2}}] $$

Motion Adds Uncertainty
---
With initial distribution $N(\mu, \sigma^2)$, and move $U$ with uncertainty $\gamma^2$, the distribution after movement is $N(\mu+U, \sigma^2+\gamma^2)$


In [21]:
# update mean and variance given 2 gaussian distribution
def update(mean1, var1, mean2, var2):
    new_mean = (var1*mean2 + var2*mean1)/(var1 + var2)
    new_var = 1./(1./var1 + 1./var2)
    return [new_mean, new_var]

# motion
def predict(mean1, var1, mean2, var2):
    new_mean = mean1 + mean2
    new_var = var1 + var2
    return [new_mean, new_var]

In [23]:
print(update(10., 8., 13., 2.))
print(predict(10., 8., 13., 2.))

[12.4, 1.6]
[23.0, 10.0]


In [45]:
def kalman_filter_1d(mu, sig, measurements, motion, measurement_sig, motion_sig):
    n = len(measurements)
    for i in range(n):
        measure = measurements[i]
        move = motion[i]
        
        mu, sig = update(mu, sig, measure, measurement_sig)
        print("estimate after measurement [{}, {}]".format(mu, sig))
        mu, sig = predict(mu, sig, move, motion_sig)
        print("estimate after movement [{}, {}]".format(mu, sig))
    return [mu, sig]

In [46]:
measurements = [5., 6., 7., 9., 10.]
motion = [1., 1., 2., 1., 1.]
measurement_sig = 4.
motion_sig = 2.
mu = 0.
sig = 10000.

kalman_filter_1d(mu, sig, measurements, motion, measurement_sig, motion_sig)

estimate after measurement [4.998000799680128, 3.9984006397441023]
estimate after movement [5.998000799680128, 5.998400639744102]
estimate after measurement [5.999200191953932, 2.399744061425258]
estimate after movement [6.999200191953932, 4.399744061425258]
estimate after measurement [6.999619127420922, 2.0951800575117594]
estimate after movement [8.999619127420921, 4.09518005751176]
estimate after measurement [8.999811802788143, 2.0235152416216957]
estimate after movement [9.999811802788143, 4.023515241621696]
estimate after measurement [9.999906177177365, 2.0058615808441944]
estimate after movement [10.999906177177365, 4.005861580844194]


[10.999906177177365, 4.005861580844194]

## Multi-Dimensional Kalman Filter
---
- **F**: state transition matrix
- **P**: covariance matrix
- $\mu$: external motion vector

### Prediction
---
$$ x' = Fx + \mu $$
$$ P' = FPF^{T} $$

In other words, the new best estimate is a prediction made from previous best estimate, plus a correction for known external influences.

And the new uncertainty is predicted from the old uncertainty, with some additional uncertainty from the environment.

### Measurement Update
---

- **Z**: measurement
- **H**: measurement transition function
- **R**: measurement noise

We have two distributions: The predicted measurement with $(\mu_0, \sigma_0) = (Hx, HPH^{T})$, and the observed measurement with $(\mu_1, \sigma_1) = (Z, R)$

Combining together, we get posterior distribution:

$$ y = Z - Hx $$
$$ S = HPH^{T} + R $$
$$ K = PH^{T}S^{-1} $$


$$ x' = x + Ky $$
$$ P' = (I - KH)P $$

the sensors operate on a state and produce a set of readings.

In [26]:
measurements = [1, 2, 3]

x = np.array([[0.], [0.]])  # initial state (location and velocity)
P = np.array([[1000., 0.], [0., 1000.]])  # initial uncertainty, covariance
u = np.array([[0.], [0.]])  # external motion
F = np.array([[1., 1.], [0, 1.]])  # next state function, state transition
H = np.array([[1., 0.]])  # measurement function, measurement transition
R = np.array([[1.]])  # measurement uncertainty
I = np.array([[1., 0.], [0., 1.]])  # identity matrix

In [49]:
# input: initial state x and covariance P
def kalman_filter(x, P):
    for i in range(len(measurements)):
        # measurement
        Z = measurements[i]  # current sense
        y = Z - np.dot(H, x)  # err between actual observation and expected observation
        S = np.dot(np.dot(H, P), np.transpose(H)) + R
        K = np.dot(np.dot(P, np.transpose(H)), np.linalg.inv(S))
        
        # posterier mu and sigma
        x = x + np.dot(K, y)
        P = np.dot((I - np.dot(K, H)), P)
        
        # predict
        x = np.dot(F, x) + u
        P = np.dot(np.dot(F, P), np.transpose(F))
    return x, P

In [16]:
x_new, P_new = kalman_filter(x, P)
print("x_new \n", x_new)
print("P_new \n", P_new)

x_new 
 [[3.99966644]
 [0.99999983]]
P_new 
 [[2.33189042 0.99916761]
 [0.99916761 0.49950058]]


### 2D State
---
initial state: $(x, y, x', y')$ --> `(loc1, loc2, vel1, vel2)`

In [50]:
measurements = np.array([[[5.], [10.]], [[6.], [8.]], [[7.], [6.]], [[8.], [4.]], [[9.], [2.]], [[10.], [0.]]])
initial_xy = np.array([4., 12.])

dt = 0.1

x = np.array([[initial_xy[0]], [initial_xy[1]], [0.], [0.]]) # initial state (location and velocity)
u = np.array([[0.], [0.], [0.], [0.]]) # external motion

P =  np.array([[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 1000, 0], [0, 0, 0, 1000]])  # initial uncertainty: 0 for positions x and y, 1000 for the two velocities
F =  np.array([[1, 0, 0.1, 0], [0, 1, 0, 0.1], [0, 0, 1, 0], [0, 0, 0, 1]])  # next state function: generalize the 2d version to 4d
H =  np.array([[1, 0, 0, 0], [0, 1, 0, 0]])  # measurement function: reflect the fact that we observe x and y but not the two velocities
R =  np.array([[0.1, 0], [0, 0.1]])  # measurement uncertainty: use 2x2 matrix with 0.1 as main diagonal
I =  np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])  # 4d identity matrix

In [51]:
x_new, P_new = kalman_filter(x, P)
print("x_new \n", x_new)
print("P_new \n", P_new)

x_new 
 [[ 11.63497546]
 [ -3.26995092]
 [ 12.7249591 ]
 [-25.4499182 ]]
P_new 
 [[0.06544265 0.         0.10907108 0.        ]
 [0.         0.06544265 0.         0.10907108]
 [0.10907108 0.         0.18178513 0.        ]
 [0.         0.10907108 0.         0.18178513]]
